# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
#nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])

In [30]:
# import libraries
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score,fbeta_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn import svm
from nltk.corpus import stopwords
from sqlalchemy import create_engine 

import warnings
warnings.filterwarnings('always', 'ResourceWarning') 

In [3]:
# load data from database
engine = create_engine('sqlite:///disasterdb2.db')
df = pd.read_sql_table('disasterdb2',con=engine)

X = df.message
Y = df.iloc[:,4:]
ycols = Y.columns.tolist()

In [4]:
#for column in Y:
#    Y[column] = Y[column].astype(int)

In [5]:
print(X.shape)
print(Y.shape)

(10222,)
(10222, 35)


### 2. Write a tokenization function to process your text data

In [44]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    #tokens = [w for w in tokens if w not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    #print(text)
    clean_tokens = []
    for token in tokens:
        #print(token)
        clean_token=lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)
    return clean_tokens

In [7]:
for message in X[:5]:
    print(message)
    tokens = tokenize(message)
    print(tokens,'\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', '-', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80-90', 'destroyed', '.', 'only', 'hospital', 'st.', 'croix', 'functioning', '.', 'needs', 'supply', 'desperately', '.'] 

says: west side of Haiti, rest of the country today and tonight
['say', ':', 'west', 'side', 'haiti', ',', 'rest', 'country', 'today', 'tonight'] 

Information about the National Palace-
['information', 'national', 'palace-'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [19]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(AdaBoostClassifier(n_estimators=10)))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [20]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f8efc198040>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(n_estimators=10)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
y_pred = pipeline.predict(X_test)

In [13]:
print(y_test.shape)
print(y_pred.shape)

(3067, 35)
(3067, 35)


confusion_mat = confusion_matrix(y_test,y_pred, labels = ycols)
accuracy=(y_test==y_pred).mean()
print("Labels:", ycols)
print("Confusion Matrix:\n", confusion_mat)
print("Accuracy:", accuracy)

In [22]:
print(classification_report(y_test, y_pred, target_names=ycols))
print("Fbeta score:", fbeta_score(y_test, y_pred, beta=2, average="weighted"))
print('Accuracy {}\n\n'.format(accuracy_score(y_test, y_pred)))

                        precision    recall  f1-score   support

               related       0.67      1.00      0.80      2057
               request       0.49      0.11      0.17      1101
                 offer       0.00      0.00      0.00         1
           aid_related       0.52      0.15      0.23      1195
          medical_help       0.50      0.01      0.01       188
      medical_products       0.00      0.00      0.00       111
     search_and_rescue       0.00      0.00      0.00        67
              security       0.00      0.00      0.00        37
              military       0.00      0.00      0.00        13
                 water       0.00      0.00      0.00       248
                  food       0.12      0.00      0.00       463
               shelter       0.11      0.00      0.01       329
              clothing       0.00      0.00      0.00        24
                 money       0.00      0.00      0.00        35
        missing_people       0.00      

/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, m

In [23]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__algorithm', 'clf__estimator__base_estimator', 'clf__estimator__learning_rate', 'clf__estimator__n_estimators', 'clf__estimator__random_state', 'clf__estimator', 'clf__n_jobs'])

### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
def build_model():
    
    pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf',MultiOutputClassifier(AdaBoostClassifier()))
        ])
    
    
    parameters = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [100, 200],
        'clf__estimator__learning_rate': [0.001, 0.01,0.1,0.2,0.5],
       }
    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv




In [27]:
model = build_model()
model.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f8efc198040>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__learning_rate': [0.001, 0.01, 0.1,
                                                           0.2, 0.5],
                         'clf__estimator__n_estimators': [100, 200],
                         'tfidf__use_idf': (True, False)})

In [28]:
print('Best Parameters:', model.best_params_)
#Best Parameters: {'tfidf__use_idf': False, 'vect__max_df': 0.75, 'vect__max_features': None, 'vect__ngram_range': (1, 2)}
#Best Parameters: {'clf__estimator__learning_rate': 0.01, 'clf__estimator__n_estimators': 200, 'tfidf__use_idf': True}

Best Parameters: {'clf__estimator__learning_rate': 0.01, 'clf__estimator__n_estimators': 200, 'tfidf__use_idf': True}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [29]:
y_pred = model.predict(X_test)
    
print(classification_report(y_test,y_pred, target_names = ycols))
print('Accuracy {}\n\n'.format(accuracy_score(y_test, y_pred)))

                        precision    recall  f1-score   support

               related       0.67      1.00      0.80      2057
               request       0.50      0.00      0.00      1101
                 offer       0.00      0.00      0.00         1
           aid_related       0.55      0.01      0.01      1195
          medical_help       0.00      0.00      0.00       188
      medical_products       0.00      0.00      0.00       111
     search_and_rescue       0.00      0.00      0.00        67
              security       0.00      0.00      0.00        37
              military       0.00      0.00      0.00        13
                 water       0.00      0.00      0.00       248
                  food       0.00      0.00      0.00       463
               shelter       0.00      0.00      0.00       329
              clothing       0.00      0.00      0.00        24
                 money       0.00      0.00      0.00        35
        missing_people       0.00      

/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
# Perform classification with MultinomialNB
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(MultinomialNB()))
])

pipeline.fit(X_train, y_train)
prediction = pipeline.predict(X_test)

# Perform classification with SVM, kernel=rbf
pipeline_rbf = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('classifier_rbf',MultiOutputClassifier(svm.SVC()))
])

pipeline_rbf.fit(X_train, y_train)
prediction_rbf = pipeline_rbf.predict(X_test)
    
# Perform classification with SVM, kernel=linear

pipeline_linear = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('classifier_rbf',MultiOutputClassifier(svm.SVC(kernel='linear')))
])

pipeline_linear.fit(X_train, y_train)
prediction_linear = pipeline_linear.predict(X_test)
# Perform classification with SVM, kernel=linear

pipeline_liblinear = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('classifier_rbf',MultiOutputClassifier(svm.LinearSVC()))
])

pipeline_liblinear.fit(X_train, y_train)
prediction_liblinear = pipeline_liblinear.predict(X_test)


In [46]:

# Print results in a nice table for MultinomialNB
#print(classification_report(y_test,y_pred, target_names = ycols))
#print('Accuracy {}\n\n'.format(accuracy_score(y_test, y_pred)))

print("\nResults for NaiveBayes (MultinomialNB) ")
print(classification_report(y_test, prediction, target_names=ycols))
print("\nAccuracy score of Multinomial naive bayes algorithm -----> " + str(accuracy_score(y_test, prediction)))

print("\n\n\n")

print("Reviews Prediction")
#print("\nPredicted label is------> "+ prediction[10]) 
#print("\nMovie Review is ------> \n"+ X_test[10])


Results for NaiveBayes (MultinomialNB) 
                        precision    recall  f1-score   support

               related       0.67      1.00      0.80      2057
               request       0.26      0.01      0.02      1101
                 offer       0.00      0.00      0.00         1
           aid_related       0.48      0.05      0.09      1195
          medical_help       0.00      0.00      0.00       188
      medical_products       0.00      0.00      0.00       111
     search_and_rescue       0.00      0.00      0.00        67
              security       0.00      0.00      0.00        37
              military       0.00      0.00      0.00        13
                 water       0.00      0.00      0.00       248
                  food       0.00      0.00      0.00       463
               shelter       0.00      0.00      0.00       329
              clothing       0.00      0.00      0.00        24
                 money       0.00      0.00      0.00        3

/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, m

In [40]:
# Print results in a nice table for SVM algorithm with rbf kernal
print("\nResults for SVM algorithm with rbf kernel")
print(classification_report(y_test, prediction_rbf,target_names=ycols))
print("\nAccuracy score of SVM algorithm with rbf kernel-----> " + str(accuracy_score(y_test, prediction_rbf)))



Results for SVM algorithm with rbf kernel
                        precision    recall  f1-score   support

               related       0.67      0.97      0.80      2057
               request       0.47      0.10      0.17      1101
                 offer       0.00      0.00      0.00         1
           aid_related       0.49      0.18      0.26      1195
          medical_help       0.00      0.00      0.00       188
      medical_products       0.00      0.00      0.00       111
     search_and_rescue       0.00      0.00      0.00        67
              security       0.00      0.00      0.00        37
              military       0.00      0.00      0.00        13
                 water       0.00      0.00      0.00       248
                  food       0.00      0.00      0.00       463
               shelter       0.00      0.00      0.00       329
              clothing       0.00      0.00      0.00        24
                 money       0.00      0.00      0.00       

/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, m

In [47]:
# Print results in a nice table for SVM algorithm with linear kernal
print("\nResults for SVM algorithm with linear kernel")
print(classification_report(y_test, prediction_linear,target_names=ycols))
print("\nAccuracy score of SVM algorithm with linear kernal-----> " + str(accuracy_score(y_test, prediction_linear)))


Results for SVM algorithm with linear kernel
                        precision    recall  f1-score   support

               related       0.68      0.97      0.80      2057
               request       0.46      0.16      0.23      1101
                 offer       0.00      0.00      0.00         1
           aid_related       0.50      0.25      0.33      1195
          medical_help       0.00      0.00      0.00       188
      medical_products       0.00      0.00      0.00       111
     search_and_rescue       0.00      0.00      0.00        67
              security       0.00      0.00      0.00        37
              military       0.00      0.00      0.00        13
                 water       0.00      0.00      0.00       248
                  food       0.00      0.00      0.00       463
               shelter       0.00      0.00      0.00       329
              clothing       0.00      0.00      0.00        24
                 money       0.00      0.00      0.00    

/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, m

In [49]:
# Print results in a nice table for SVM algorithm with liblinear kernal
print("\nResults for SVM algorithm with liblinear kernel")
print(classification_report(y_test, prediction_liblinear,target_names=ycols))
print("\nAccuracy score of SVM algorithm with liblinear kernal-----> " + str(accuracy_score(y_test, prediction_liblinear)))



Results for SVM algorithm with liblinear kernel
                        precision    recall  f1-score   support

               related       0.69      0.82      0.75      2057
               request       0.42      0.27      0.33      1101
                 offer       0.00      0.00      0.00         1
           aid_related       0.44      0.32      0.37      1195
          medical_help       0.00      0.00      0.00       188
      medical_products       0.00      0.00      0.00       111
     search_and_rescue       1.00      0.01      0.03        67
              security       0.00      0.00      0.00        37
              military       0.00      0.00      0.00        13
                 water       0.38      0.01      0.02       248
                  food       0.18      0.03      0.06       463
               shelter       0.21      0.02      0.03       329
              clothing       0.00      0.00      0.00        24
                 money       0.00      0.00      0.00 

/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thimothekonchou/opt/anaconda3/envs/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, m

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.